In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import models
import torchvision.transforms as T
import torchvision.datasets as dset

from torch.optim import lr_scheduler

from torch.utils.data import DataLoader, sampler, Dataset
from pytorch_load_data import load_data

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import copy

plt.ion()   # interactive mode

In [2]:
dataloaders, poster_train, poster_val, poster_test = load_data(64)
# print(iter(dataloaders['train']).next()[0].size())

In [3]:
poster_train[0][1]

[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# # functions to show an image
# def imshow(img):
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))

# # get some random training images
# dataiter = iter(dataloaders['train'])
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(labels)
# # print(' '.join('%5s' % labels for j in range(BATCHSIZE)))

In [6]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [7]:
model_conv = torchvision.models.resnet18(pretrained=True)

for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 19)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.1, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [63]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
#                 scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):

                    scores = model(inputs)
                    loss = multilabel_Loss(scores, labels)
                    
        
#                     outputs = model(inputs)
#                     _, preds = torch.max(outputs, 1)
#                     loss = criterion(outputs, labels)
#                     print("test")
#                     print(type(loss))
    
    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)

#             epoch_loss = running_loss / dataset_sizes[phase]
#             epoch_acc = running_corrects.double() / dataset_sizes[phase]

#             print('{} Loss: {:.4f} Acc: {:.4f}'.format(
#                 phase, epoch_loss, epoch_acc))

#             # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

#         print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [67]:
def multilabel_Loss(x, y):
    
##numpy version    
#     x = x.data.numpy()
#     shifted_logits = x - np.max(x, axis=1, keepdims=True)    
#     Z = np.sum(np.exp(shifted_logits), axis=1, keepdims=True)    
#     log_probs = shifted_logits - np.log(Z)    
#     probs = np.exp(log_probs)
#     N, C = x.shape
    
#     y_arr = np.zeros((N, C))
#     for idx,each in enumerate(y):
#         y_arr[:,idx] = each.numpy()
    
#     num_labels = y_arr.sum(axis=1)    
#     log_labels = log_probs.T/num_labels
#     loss = np.sum(log_labels) / 19

    ## to calculate the log of the 
    shifted_logits = x - torch.max(x, dim=1, keepdim=True)[0]
    summed = torch.sum(torch.exp(shifted_logits), dim=1, keepdim=True)    
    log_probs = torch.log(torch.exp(shifted_logits) / summed)
#     log_probs = torch.log(probs)
#     log_probs = shifted_logits - torch.log(Z)    
#     probs = torch.exp(log_probs)
    N, C = x.size()
    
    ##the labels come in as a list of tensors, so need to be converted to a single tensor
    y_arr = torch.zeros((N, C))
    for idx,each in enumerate(y):
        y_arr[:,idx] = each
    
    num_labels = torch.sum(y_arr, dim=1)    
    log_labels = torch.transpose(log_probs,1,0)/num_labels
    loss = torch.sum(log_labels) / C

    return loss #, dx

In [68]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=2)

Epoch 0/1
----------


KeyboardInterrupt: 